Importation of packages

In [2]:
import filter_data  # class containing filter methods 
import date_handling  # ￼class containing date related methods
import tweet_analysis  # class containing the method for the content analysis of the tweets. 
import constants  # import global constants
import plot # plotting functions 
import dictionnaries # defined dictionaries
import numpy as np # numpy for array handling
import percentage # compute percentage

# Reload each module
import importlib

importlib.reload(filter_data)
importlib.reload(date_handling)
importlib.reload(tweet_analysis)
importlib.reload(constants)
importlib.reload(plot)
importlib.reload(dictionnaries)



<module 'dictionnaries' from '/home/loris/Desktop/populist_russians_bots/dictionnaries.py'>

Definition of constants 


In [3]:
START_DATE = np.datetime64('2014-07-01') # Date of the first published tweet in English 
END_DATE = np.datetime64('2018-06-01') # Date of the last published tweet in English 


Filter the data : 
- Keep only tweets written in english 
- Remove links in the tweets
- Remove columns that we do not need 
- Reformat dates

In [10]:
# Warning: This cell needs to be run only once. 

# Define the set of column indices to keep
COLUMN_INDICES = {1, 2, 5, 13}

# Call the function with the input file name and column indices
for i in range(1, 14): 
    input_path= constants.RAW_DATA_PATH.format(i)
    output_path= constants.PRE_PROCESSED_DATA_PATH.format(i)
    filter_data.filter_csv(input_path, output_path, COLUMN_INDICES, 4, "English") # Keep only tweets in english 
    filter_data.extract_date_columns(output_path, output_path)
    filter_data.remove_links(output_path, output_path)
    filter_data.convert_to_lowercase(output_path, output_path)

Get the array of dates that span the period of the tweets

In [4]:
date_array = date_handling.get_date_array(START_DATE, END_DATE)

(Optional) Plot the number of tweets over the period

In [7]:
nb_of_tweets_per_day = tweet_analysis.get_nb_of_tweets_per_day(date_array)
print(nb_of_tweets_per_day.sum(axis=0))


2116719.0


In [13]:
%matplotlib qt

plot.plot_result(date_array, nb_of_tweets_per_day, START_DATE, END_DATE)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

qt.qpa.wayland: Failed to initialize EGL display 3001


Dictionary analysis


In [14]:
nb_of_tweets_filtered_with_dic = tweet_analysis.dataset_analysis(dictionnaries.dic_rooduijn_pauwels, date_array)


Plot result of dictionary analysis

In [15]:
plot.plot_result(date_array,nb_of_tweets_filtered_with_dic, START_DATE, END_DATE)

Get percentage of tweets with at least one words over the total nb of tweets

In [16]:
percentage = percentage.percentage_of_tweets(nb_of_tweets_filtered_with_dic, nb_of_tweets_per_day)
plot.plot_result(date_array, percentage, START_DATE, END_DATE)

# Populist dictionaries 


Extract the dictionaries of populist expressions

In [5]:
anti_elitism_dictionary = dictionnaries.extract_grundle_translation_dictionary("anti-elitism")
people_centrism_dictionary = dictionnaries.extract_grundle_translation_dictionary("people-centrism")
sovereignty_dictionary = dictionnaries.extract_grundle_translation_dictionary("sovereignty")
print("anti-elitism dictionary contains {} words".format(len(anti_elitism_dictionary)))
print("people_centrism dictionary contains {} words".format(len(people_centrism_dictionary)))
print("sovereignty dictionary contains {} words".format(len(sovereignty_dictionary)))

anti-elitism dictionary contains 7288 words
people_centrism dictionary contains 3135 words
sovereignty dictionary contains 426 words


# Analysis

Perform the anti-elitism analysis 

In [5]:
nb_of_tweets_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary,date_array)
np.save('./cached_results/nb_of_tweets_anti_elitist.npy', nb_of_tweets_anti_elitist) # Store result in storage

processing file 1
4.9114227294921875e-05
processing file 2
131.72613978385925
processing file 3
257.0437738895416
processing file 4
395.72542905807495
processing file 5
521.0596373081207
processing file 6
629.6913123130798
processing file 7
771.8664197921753
processing file 8
872.5399751663208
processing file 9
953.3470366001129
processing file 10
1032.5981602668762
processing file 11
1146.304619550705
processing file 12
1268.4046595096588
processing file 13
1380.129861831665


Perform the people_centrism analysis 


In [13]:
nb_of_tweets_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary,date_array)
np.save('./cached_results/nb_of_tweets_people_centrism.npy', nb_of_tweets_people_centrism)

processing file 1
8.440017700195312e-05
processing file 2
60.74401068687439
processing file 3
119.36048793792725
processing file 4
183.60057997703552
processing file 5
241.99017453193665
processing file 6
290.83711409568787
processing file 7
352.8362989425659
processing file 8
397.9904148578644
processing file 9
434.26134753227234
processing file 10
469.16220331192017
processing file 11
518.4147465229034
processing file 12
572.2147982120514
processing file 13
622.5426344871521


Perform the sovereignty analysis

In [14]:
nb_of_tweets_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array)
np.save('./cached_results/nb_of_tweets_sovereignty.npy', nb_of_tweets_sovereignty)

processing file 1
0.0002989768981933594
processing file 2
8.894880056381226
processing file 3
17.40390396118164
processing file 4
26.74119734764099
processing file 5
35.13206362724304
processing file 6
42.358131408691406
processing file 7
51.51526498794556
processing file 8
58.053232192993164
processing file 9
63.371689796447754
processing file 10
68.62022280693054
processing file 11
75.86168646812439
processing file 12
83.84404349327087
processing file 13
91.11701703071594


Perform the analysis for the Right Trolls 

In [6]:
nb_tweets_right_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary, date_array, "RightTroll")
nb_tweets_right_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary, date_array, "RightTroll")
nb_tweets_right_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array, "RightTroll")
np.save("./cached_results/nb_tweets_right_anti_elitist.npy", nb_tweets_right_anti_elitist)
np.save("./cached_results/nb_tweets_right_people_centrism.npy", nb_tweets_right_people_centrism)
np.save("./cached_results/nb_tweets_right_sovereignty.npy", nb_tweets_right_sovereignty)

processing file 1
0.0004076957702636719
processing file 2
215.3645522594452
processing file 3
411.87449622154236
processing file 4
635.8055195808411
processing file 5
844.0977220535278
processing file 6
1023.1581933498383
processing file 7
1247.3113045692444
processing file 8
1402.2921578884125
processing file 9
1526.9980773925781
processing file 10
1645.093108177185
processing file 11
1827.134846687317
processing file 12
2019.8090846538544
processing file 13
2190.700097799301
processing file 1
6.532669067382812e-05
processing file 2
97.64743971824646
processing file 3
193.1334023475647
processing file 4
296.4744896888733
processing file 5
391.2334864139557
processing file 6
469.1361665725708
processing file 7
567.8150594234467
processing file 8
641.1044006347656
processing file 9
700.5750844478607
processing file 10
757.2609882354736
processing file 11
837.755576133728
processing file 12
925.376757144928
processing file 13
1007.297886133194
processing file 1
8.0108642578125e-05
proces

Perform the analysis for the Left Trolls 

In [ ]:
nb_tweets_left_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary, date_array, "LeftTroll")
nb_tweets_left_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary, date_array, "LeftTroll")
nb_tweets_left_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array, "LeftTroll")
np.save("./cached_results/nb_tweets_left_anti_elitist.npy", nb_tweets_left_anti_elitist)
np.save("./cached_results/nb_tweets_left_people_centrism.npy", nb_tweets_left_people_centrism)
np.save("./cached_results/nb_tweets_left_sovereignty.npy", nb_tweets_left_sovereignty)

Perform the analysis for the News Feed

In [ ]:
nb_tweets_news_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary, date_array, "NewsFeed")
nb_tweets_news_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary, date_array, "NewsFeed")
nb_tweets_news_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array, "NewsFeed")
np.save("./cached_results/nb_tweets_news_anti_elitist.npy", nb_tweets_news_anti_elitist)
np.save("./cached_results/nb_tweets_news_people_centrism.npy", nb_tweets_news_people_centrism)
np.save("./cached_results/nb_tweets_news_sovereignty.npy", nb_tweets_news_sovereignty)

Perform the analysis for the hashtag gamer

In [ ]:
nb_tweets_gamer_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary, date_array, "HashtagGamer")
nb_tweets_gamer_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary, date_array, "HashtagGamer")
nb_tweets_gamer_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array, "HashtagGamer")
np.save("./cached_results/nb_tweets_gamer_anti_elitist.npy", nb_tweets_gamer_anti_elitist)
np.save("./cached_results/nb_tweets_gamer_people_centrism.npy", nb_tweets_gamer_people_centrism)
np.save("./cached_results/nb_tweets_gamer_sovereignty.npy", nb_tweets_gamer_sovereignty)

Perform the analysis for the Fearmonger

In [ ]:
nb_tweets_fear_anti_elitist = tweet_analysis.dataset_analysis(anti_elitism_dictionary, date_array, "Fearmonger")
nb_tweets_fear_people_centrism = tweet_analysis.dataset_analysis(people_centrism_dictionary, date_array, "Fearmonger")
nb_tweets_fear_sovereignty = tweet_analysis.dataset_analysis(sovereignty_dictionary, date_array, "Fearmonger")
np.save("./cached_results/nb_tweets_fear_anti_elitist.npy", nb_tweets_fear_anti_elitist)
np.save("./cached_results/nb_tweets_fear_people_centrism.npy", nb_tweets_fear_people_centrism)
np.save("./cached_results/nb_tweets_fear_sovereignty.npy", nb_tweets_fear_sovereignty)

# Plot results 

Global anti-elitist 

In [ ]:
%matplotlib qt
# %matplotlib inline 
nb_of_tweets_anti_elitist = np.load('./cached_results/nb_of_tweets_anti_elitist.npy')
percentage_anti_elitist = percentage.percentage_of_tweets(nb_of_tweets_anti_elitist, nb_of_tweets_per_day)
plot.plot_result(date_array, percentage_anti_elitist, START_DATE, END_DATE)
mean_percentage = np.mean(percentage_anti_elitist)

print("Mean percentage: ", mean_percentage)

Global people centrism


In [ ]:
%matplotlib qt
# %matplotlib inline 
nb_of_tweets_people_centrism = np.load('./cached_results/nb_of_tweets_people_centrism.npy')
percentage_people_centrism = percentage.percentage_of_tweets(nb_of_tweets_people_centrism, nb_of_tweets_per_day)
plot.plot_result(date_array, percentage_people_centrism, START_DATE, END_DATE)
mean_percentage = np.mean(percentage_people_centrism)

print("Mean percentage: ", mean_percentage)

Global sovereignty
